In [40]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import sqlite3
import numpy as np
from utils import connect_db

'''
TABLE packages:
id INTEGER PRIMARY KEY, name TEXT, maintainer TEXT, version TEXT, first_uploaded TEXT, was_suspicious_before INTEGER

TABLE package_info
id INTEGER PRIMARY KEY, package_id INTEGER, num_files INTEGER, total_additions INTEGER, total_deletions INTEGER, links INTEGER, avg_dir_depth REAL, max_dir_depth INTEGER, avg_file_size REAL, median_file_size REAL, total_size INTEGER, has_executables INTEGER, commit_hash TEXT, time_since_last_commit INTEGER, upload_time TEXT, dependency_count INTEGER
'''

train_db = 'package_analysis.db'

def import_data():
    conn, c = connect_db(train_db)
    df_packages = pd.read_sql_query("SELECT * FROM packages", conn)
    df_package_info = pd.read_sql_query("SELECT * FROM package_info", conn)
    conn.close()
    return df_packages, df_package_info

def split_data(df_packages, df_package_info):
    # y = []
    # for row in df_package_info.iterrows():
    #     y.append(df_packages.loc[df_packages['id'] == row[1]['package_id']]['was_suspicious_before'].values[0])
    # y = np.array(y)
    # X_train, X_test, y_train, y_test = train_test_split(df_package_info, y, test_size=0.3, random_state=42)
    X_size = df_packages.shape[0]
    # Sample 70% of the data for training
    X_sample = df_packages.sample(frac=0.7)
    # Get the package info for the sampled packages
    X_train = df_package_info.loc[df_package_info['package_id'].isin(X_sample['id'])]
    y_train = []
    for row in X_train.iterrows():
        y_train.append(df_packages.loc[df_packages['id'] == row[1]['package_id']]['was_suspicious_before'].values[0])
    y_train = np.array(y_train)
    # Get the package info for the remaining packages
    X_test = df_package_info.loc[~df_package_info['package_id'].isin(X_sample['id'])]
    y_test = []
    for row in X_test.iterrows():
        y_test.append(df_packages.loc[df_packages['id'] == row[1]['package_id']]['was_suspicious_before'].values[0])
    return X_train, X_test, y_train, y_test

def train_model(X_train, y_train):
    model = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
    model.fit(X_train, y_train)
    return model

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy



In [41]:
# def main():
    # df_packages, df_package_info = import_data()
    
    # print(df_packages.head(5))
    # print(df_package_info.head())
    # X_train, X_test, y_train, y_test = split_data(df_packages, df_package_info)
    # model = train_model(X_train, y_train)
    # accuracy = evaluate_model(model, X_test, y_test)
    # print(f'Accuracy: {accuracy}')
    
# if __name__ == '__main__':
#     main()

In [42]:
df_packages, df_package_info = import_data()
df_packages

,id,name,maintainer,version,first_uploaded,was_suspicious_before
0,1,fight,Belinda Morrison,4.8.0,2021-10-04 15:22:36.170536,0
1,2,material,Julia Franco,7.9.5,2020-03-08 21:56:56.466346,0
2,3,professional,Shirley Scott,7.7.3,2020-05-16 04:52:07.904179,0
3,4,try,Michael Marshall,7.4.0,2021-08-18 14:28:27.758953,0
4,5,entire,David Wagner,7.3.0,2020-08-16 00:44:18.554832,0
...,...,...,...,...,...,...
995,996,none,Stephanie Blackwell,0.0.4,2023-07-11 07:02:37.533220,0
996,997,become,Thomas Thomas,6.8.5,2023-05-31 19:07:19.067807,0
997,998,decide,Jacob Cunningham,6.2.0,2022-03-28 19:33:55.635473,0
998,999,guess,Jonathan Lara,1.8.0,2024-08-16 00:16:28.777249,0


In [43]:
df_package_info

,id,package_id,num_files,total_additions,total_deletions,links,avg_dir_depth,max_dir_depth,avg_file_size,median_file_size,total_size,has_executables,commit_hash,time_since_last_commit,upload_time,dependency_count
0,1,1,95,1209,0,36,5.0,7,820.0,402.0,77900,0,b1e9a063d0fa0ad7466b4dc4010dfa64178807d1607188...,0,2021-10-04 15:22:36.170536,33
1,2,1,94,273,158,35,5.0,7,857.0,383.0,81415,0,47d6817de87fb7c09551b9f94a723193fd874723e2ecd7...,61931,2021-10-05 08:34:47.170536,33
2,3,1,94,15,4,34,5.0,7,861.0,356.0,77490,0,d82273c28731542065c628c0a17784456ea5769190bc07...,1105359,2021-10-18 03:37:26.170536,33
3,4,1,93,60,33,32,5.0,7,860.0,356.0,81700,0,25c6d4737f9d49354c0d8c300465a7d29c63cf09c8eff3...,363069,2021-10-22 08:28:35.170536,31
4,5,1,97,133,12,32,5.0,7,871.0,350.0,84487,0,e75388c138de86c5abb21dfb851c601d3656a05d70c645...,2123597,2021-11-15 22:21:52.170536,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93591,93592,947,109,135,47,28,7.0,9,379.0,620.0,41311,0,afc9a507385a64cf614bb8209d18357157202a5868a705...,2644628,2024-06-30 06:46:13.463615,71
93592,93593,947,109,240,144,30,7.0,9,389.0,614.0,42401,0,e3abd6e54c6107f87b0766b9ea379b06b3ee5a8dde271f...,968017,2024-07-11 11:39:50.463615,77
93593,93594,947,109,347,217,31,7.0,9,339.0,600.0,38307,0,acaa34206fb133d2fbcb1dbd7f35f2c1547ebc3494a6a2...,1312020,2024-07-26 16:06:50.463615,73
93594,93595,947,112,6,0,30,7.0,9,339.0,589.0,33900,0,d5a842de5663230ed4c94df5eaee068218e05faf9e3437...,2571957,2024-08-25 10:32:47.463615,68


In [ ]:
# Create a new dataframe with the package info and the label then print to a csv file
for row in df_package_info.iterrows():
    df_package_info.loc[row[0], 'was_suspicious_before'] = df_packages.loc[df_packages['id'] == row[1]['package_id']]['was_suspicious_before'].values[0]

df_package_info.to_csv('package_info.csv', index=False)

In [44]:
X_train, X_test, y_train, y_test = split_data(df_packages, df_package_info)
# model = train_model(X_train, y_train)
# accuracy = evaluate_model(model, X_test, y_test)
# print(f'Accuracy: {accuracy}')

In [45]:
X_train

,id,package_id,num_files,total_additions,total_deletions,links,avg_dir_depth,max_dir_depth,avg_file_size,median_file_size,total_size,has_executables,commit_hash,time_since_last_commit,upload_time,dependency_count
0,1,1,95,1209,0,36,5.0,7,820.0,402.0,77900,0,b1e9a063d0fa0ad7466b4dc4010dfa64178807d1607188...,0,2021-10-04 15:22:36.170536,33
1,2,1,94,273,158,35,5.0,7,857.0,383.0,81415,0,47d6817de87fb7c09551b9f94a723193fd874723e2ecd7...,61931,2021-10-05 08:34:47.170536,33
2,3,1,94,15,4,34,5.0,7,861.0,356.0,77490,0,d82273c28731542065c628c0a17784456ea5769190bc07...,1105359,2021-10-18 03:37:26.170536,33
3,4,1,93,60,33,32,5.0,7,860.0,356.0,81700,0,25c6d4737f9d49354c0d8c300465a7d29c63cf09c8eff3...,363069,2021-10-22 08:28:35.170536,31
4,5,1,97,133,12,32,5.0,7,871.0,350.0,84487,0,e75388c138de86c5abb21dfb851c601d3656a05d70c645...,2123597,2021-11-15 22:21:52.170536,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93541,93542,104,77,358,227,27,1.0,6,100.0,551.0,7100,0,054b85b5053b71b4a4d4fc34162c44fcc04eac51b6a01f...,9487110,2024-02-19 00:03:53.272468,12
93542,93543,104,77,8,1,26,1.0,6,109.0,548.0,8393,0,6e597fb1a9fef016e1aa529e765718802eda4e2b0d2d0f...,7225768,2024-05-12 15:13:21.272468,12
93543,93544,104,79,236,62,26,1.0,6,110.0,519.0,9130,0,acd762a5840bce63b131da724fdfa4798acdf63105325c...,10857903,2024-09-15 07:18:24.272468,12
93544,93545,104,78,126,74,27,1.0,6,107.0,557.0,8025,0,b43921d8222cf0ac620aa8386716794711472f4483108e...,5461833,2024-11-17 12:28:57.272468,12


In [ ]:
X_train[df_package_info['package_id'] == 1]
y_train[

/tmp/ipykernel_190273/2961567106.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_train[df_package_info['package_id'] == 1]


,id,package_id,num_files,total_additions,total_deletions,links,avg_dir_depth,max_dir_depth,avg_file_size,median_file_size,total_size,has_executables,commit_hash,time_since_last_commit,upload_time,dependency_count
0,1,1,95,1209,0,36,5.0,7,820.0,402.0,77900,0,b1e9a063d0fa0ad7466b4dc4010dfa64178807d1607188...,0,2021-10-04 15:22:36.170536,33
1,2,1,94,273,158,35,5.0,7,857.0,383.0,81415,0,47d6817de87fb7c09551b9f94a723193fd874723e2ecd7...,61931,2021-10-05 08:34:47.170536,33
2,3,1,94,15,4,34,5.0,7,861.0,356.0,77490,0,d82273c28731542065c628c0a17784456ea5769190bc07...,1105359,2021-10-18 03:37:26.170536,33
3,4,1,93,60,33,32,5.0,7,860.0,356.0,81700,0,25c6d4737f9d49354c0d8c300465a7d29c63cf09c8eff3...,363069,2021-10-22 08:28:35.170536,31
4,5,1,97,133,12,32,5.0,7,871.0,350.0,84487,0,e75388c138de86c5abb21dfb851c601d3656a05d70c645...,2123597,2021-11-15 22:21:52.170536,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,131,1,75,23,2,13,5.0,7,956.0,396.0,78392,0,f886f32cd0bff43531e096037184402329a0e690b0eef3...,927704,2024-10-19 08:35:25.170536,20
131,132,1,70,28,9,13,5.0,7,919.0,411.0,63411,0,1da35e80a2bd4a053d170297afa317209ebb862b46ca0a...,904904,2024-10-29 19:57:09.170536,20
132,133,1,68,87,11,13,5.0,7,953.0,418.0,69569,0,9e330a945e50731541f34bfc59f9c2260f98b1dd50d6c8...,1239682,2024-11-13 04:18:31.170536,20
133,134,1,77,4,2,13,5.0,7,952.0,429.0,63784,0,c49e96084f92d4f02d8f802d849b73a57c3cbe91a1c8f1...,1248070,2024-11-27 14:59:41.170536,20
